# Import Required Libraries
Import the necessary libraries, including pandas and selenium.

In [1]:
# Import Required Libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Set Up WebDriver
Initialize the Selenium WebDriver and configure it for the desired browser.

In [2]:
# Set up WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

# Navigate to the Web Page
Use the WebDriver to navigate to the target web page containing the table.

https://gestao.granitopagamentos.com.br/Login/Index

sandro.leao@creditoessencial.com.br

1011S@ndr0310

# Define Helper Functions
Functions to interact with the web page, such as changing tabs, extracting data, and clicking table rows.

In [16]:
# Function to change to the 'NL/Taxas' tab
def change_to_nl_taxas():
    try:
        nl_taxas_tab = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.LINK_TEXT, 'NL / Taxas'))
        )
        nl_taxas_tab.click()
    except Exception as e:
        print(f'Error changing to the "NL / Taxas" tab: {e}')

# Function to switch back to the 'Pesquisa' tab
def switch_to_pesquisa_tab():
    try:
        pesquisa_tab = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.LINK_TEXT, 'Pesquisa'))
        )
        pesquisa_tab.click()
    except Exception as e:
        print(f'Error switching to the "Pesquisa" tab: {e}')

# Function to extract tax rates
def get_tax_rates():
    try:
        taxas_table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//table[@id='tabelaTaxas']"))
        )
        rows = taxas_table.find_elements(By.TAG_NAME, 'tr')
        taxas = []
        for row in rows:
            cols = row.find_elements(By.TAG_NAME, 'td')
            taxas.append([col.text for col in cols])
        return taxas
    except Exception as e:
        print(f'Error extracting tax rates: {e}')
        return []

# Function to click a table row
def click_table_row():
    try:
        table_row = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//table[@id='tabelaClientes']/tbody/tr"))
        )
        table_row.click()
    except Exception as e:
        print(f'Error clicking table row: {e}')

# Function to search by CPF/CNPJ
def search_by_cpf_cnpj(cpf_cnpj):
    try:
        cpf_cnpj_input = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.ID, 'CPF_CNPJPesquisa'))
        )
        cpf_cnpj_input.clear()
        cpf_cnpj_input.send_keys(cpf_cnpj)

        search_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'btnPesquisarCliente'))
        )
        search_button.click()
    except Exception as e:
        print(f'Error searching by CPF/CNPJ {cpf_cnpj}: {e}')

# Function to extract tax rates by IDs
def get_tax_rates_by_ids():
    ids = [
        "TaxaDebitoVisa",
        "TaxaCreditoVisa",
        "TaxaCredito2a6Visa",
        "TaxaCredito7a12Visa",
        "TaxaDebitoMastercard",
        "TaxaCreditoMastercard",
        "TaxaCredito2a6Mastercard",
        "TaxaCredito7a12Mastercard",
        "TaxaDebitoElo",
        "TaxaCreditoElo",
        "TaxaCredito2a6Elo",
        "TaxaCredito7a12Elo",
        "TaxaCreditoAmericanExpress",
        "TaxaCredito2a6AmericanExpress",
        "TaxaCredito7a12AmericanExpress",
        "TaxaCreditoHipercard",
        "TaxaCredito2a6Hipercard",
        "TaxaCredito7a12Hipercard"
    ]

    taxas = {}
    for id_ in ids:
        try:
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, id_))
            )
            taxas[id_] = element.get_attribute("value")
        except Exception as e:
            taxas[id_] = "Não encontrado"
            print(f"Erro ao buscar o ID {id_}: {e}")
    return taxas

# Read the Spreadsheet
Load the CPF/CNPJ data from the Excel file.

In [5]:
# Read the spreadsheet
cpf_cnpj_df = pd.read_excel('cpf_cnpj_transacionado.xlsx')
cpf_cnpj_data = cpf_cnpj_df['cpf_cnpj']

# Process Each CPF/CNPJ
Iterate through the CPF/CNPJ data, perform searches, click table rows, and extract tax rates.

In [ ]:
# Initialize a dictionary to store results
taxas_resultado = {'cpf_cnpj': [], 'taxas': []}

# Loop through each CPF/CNPJ
for cpf_cnpj in cpf_cnpj_data:
    # Switch to the 'Pesquisa' tab
    switch_to_pesquisa_tab()

    # Search for the current CPF/CNPJ
    search_by_cpf_cnpj(cpf_cnpj)
    time.sleep(2)  # Wait for the page to load

    # Click the corresponding table row
    click_table_row()

    # Switch to the 'NL/Taxas' tab
    change_to_nl_taxas()

    # Extract tax rates by IDs
    taxas = get_tax_rates_by_ids()

    # Append results to the dictionary
    taxas_resultado['cpf_cnpj'].append(cpf_cnpj)
    taxas_resultado['taxas'].append(taxas)

In [23]:
# Transform the results into a DataFrame with tax IDs as columns
result_data = []
for i, cpf_cnpj in enumerate(taxas_resultado['cpf_cnpj']):
    # Create a row with CPF/CNPJ and corresponding tax rates
    row = {'cpf_cnpj': cpf_cnpj}
    for key, value in taxas_resultado['taxas'][i].items():
        # Convert percentage strings to numeric values
        if value.endswith('%'):
            row[key] = float(value.replace(',', '.').replace(' %', '')) / 100
        else:
            row[key] = None  # Handle missing or invalid values
    result_data.append(row)

# Create a DataFrame from the transformed data
result_df = pd.DataFrame(result_data)

# Save Results
Save the extracted data to a CSV file.

In [29]:
# Save the results to a CSV file
result_df.to_excel('taxas_resultado.xlsx', index=False)

# Close the WebDriver
# driver.quit()

In [26]:
result_df

,cpf_cnpj,TaxaDebitoVisa,TaxaCreditoVisa,TaxaCredito2a6Visa,TaxaCredito7a12Visa,TaxaDebitoMastercard,TaxaCreditoMastercard,TaxaCredito2a6Mastercard,TaxaCredito7a12Mastercard,TaxaDebitoElo,TaxaCreditoElo,TaxaCredito2a6Elo,TaxaCredito7a12Elo,TaxaCreditoAmericanExpress,TaxaCredito2a6AmericanExpress,TaxaCredito7a12AmericanExpress,TaxaCreditoHipercard,TaxaCredito2a6Hipercard,TaxaCredito7a12Hipercard
0,088.334.376-20,0.0160,0.0325,0.0352,0.0369,0.0160,0.0325,0.0352,0.0369,0.0218,0.0340,0.0410,0.0450,0.0404,0.0419,0.0347,0.0385,0.0390,0.0379
1,029.037.156.29,0.0160,0.0325,0.0352,0.0369,0.0160,0.0325,0.0352,0.0369,0.0218,0.0340,0.0410,0.0450,0.0404,0.0419,0.0347,0.0385,0.0390,0.0379
2,020.376.601-32,0.0160,0.0325,0.0352,0.0369,0.0160,0.0325,0.0352,0.0369,0.0218,0.0340,0.0410,0.0450,0.0404,0.0419,0.0347,0.0385,0.0390,0.0379
3,54.112.002/0001-00,0.0196,0.0285,0.0395,0.0475,0.0196,0.0285,0.0423,0.0481,0.0228,0.0358,0.0451,0.0532,0.0358,0.0451,0.0532,0.0358,0.0451,0.0532
4,130.448.866-78,0.0196,0.0285,0.0395,0.0475,0.0196,0.0285,0.0423,0.0481,0.0228,0.0358,0.0451,0.0532,0.0358,0.0451,0.0532,0.0358,0.0451,0.0532
5,051.930.126-98,0.0160,0.0325,0.0352,0.0369,0.0160,0.0325,0.0352,0.0369,0.0218,0.0340,0.0410,0.0450,0.0404,0.0419,0.0347,0.0385,0.0390,0.0379
6,955.936.166-04,0.0160,0.0325,0.0352,0.0751,0.0160,0.0325,0.0352,0.0369,0.0218,0.0340,0.0410,0.0450,0.0404,0.0419,0.0347,0.0385,0.0390,0.0379
7,388.727.143-20,0.0196,0.0325,0.0423,0.0481,0.0196,0.0325,0.0423,0.0481,0.0228,0.0358,0.0451,0.0532,0.0384,0.0451,0.0532,0.0358,0.0451,0.0532
8,040.525.196-39,0.0160,0.0325,0.0352,0.0369,0.0160,0.0325,0.0352,0.0369,0.0218,0.0340,0.0410,0.0450,0.0384,0.0404,0.0419,0.0347,0.0385,0.0390
9,095.003.396-09,0.0196,0.0285,0.0423,0.0481,0.0196,0.0285,0.0423,0.0481,0.0228,0.0358,0.0451,0.0532,0.0358,0.0451,0.0532,0.0358,0.0451,0.0532
